In [ ]:
import numpy as np
import time
from scipy.sparse import issparse, csc_matrix
import datetime

class SparseCoordinateDescentSVM_2:
    def __init__(self, C=1.0, max_iter=10000, tol=1e-8, sigma=0.01, beta=0.5, verbose=True):
        self.C = C
        self.max_iter = max_iter
        self.tol = tol
        self.sigma = sigma
        self.gamma = 0
        self.beta = beta
        self.verbose = verbose
        self.w = None
        self.z = None
        self.k = 0

        self.lambdas = {}
        self.times = []
        self.relative_diffs = []
        self.obj_values =[]
        self.time_start = time.time()
    def _precompute_H(self, X):
        """
            Precompute diagonal elements of Hessian matrix:
            H_i = 1 + 2C * sum_j x_ji^2
            Used for second derivatives during coordinate updates.
        """
        return  1 + 2 * self.C * (X.power(2).sum(axis=0)).A1
    def _d_prime_i_0(self, X, y, i):
        margins = 1 - y * self.z
        active = margins > 0

        # Indeksy kolumny i w formacie CSC
        col_start = X.indptr[i]
        col_end = X.indptr[i + 1]
        row_indices = X.indices[col_start:col_end]
        data = X.data[col_start:col_end]

        # Maska aktywnych przykładów w tych wierszach
        mask = active[row_indices]

        if not np.any(mask):
            return self.w[i]  # brak aktywnych przykładów, gradient to tylko w[i]

        filtered_y = y[row_indices][mask]
        filtered_margins = margins[row_indices][mask]
        filtered_data = data[mask]

        gradient_sum = np.sum(filtered_data * filtered_y * filtered_margins)
        return self.w[i] - 2 * self.C * gradient_sum


    def _d_double_prime_i_0(self, X, y, i):
        # Wyciągamy niezerowe elementy kolumny i z csc_matrix X
        col_start = X.indptr[i]
        col_end = X.indptr[i + 1]
        indices = X.indices[col_start:col_end]  # wiersze niezerowych elementów kolumny i
        data = X.data[col_start:col_end]       # wartości tych elementów

        # Obliczamy marginesy
        margins = 1 - y * self.z
        # Tworzymy maskę aktywnych przykładów (tych, które łamią warunek margin > 0)
        active_mask = margins[indices] > 0

        if not np.any(active_mask):
            return 1.0  # tylko regularizacja, brak strat

        data_active = data[active_mask]

        return 1.0 + 2 * self.C * np.sum(data_active ** 2)


    def _newton_direction(self, X, y, i,  denominator = None):

        numerator = self._d_prime_i_0(X, y, i)

        if (denominator == None):
            denominator = self._d_double_prime_i_0(X, y, i)
        return -numerator / denominator if denominator != 0 else 0.0
    def _d_i_z(self, X, y, i, z):
        x_col = X[:, i]
        indices = x_col.indices
        delta = z * x_col.data
        z_new_part = self.z[indices] + delta
    
        margins_part = 1 - y[indices] * z_new_part
        active = margins_part > 0
        loss_term = np.sum(margins_part[active] ** 2)
    
        w_norm_sq = np.dot(self.w, self.w) + 2 * z * self.w[i] + z**2
        return 0.5 * w_norm_sq + self.C * loss_term


    def _compute_threshhold_lambda(self, X, y, i):
  
        dii = self._d_double_prime_i_0(X, y, i)

        return dii / (0.5 * self.H[i]+ self.sigma), dii



    def _compute_lambda(self, X, y, i, d):
        lambda_bar = self.lambdas[i]
        if abs(d) < 1e-12:
            return 0.0
        if 1.0 <= lambda_bar:
            return 1.0

        # Otherwise, perform line search
        D0 = self._d_i_z(X, y, i, 0)
        k = 0
        while True:
            lam = self.beta ** k
            z = lam * d
            Dz = self._d_i_z(X, y, i, z)
            if Dz - D0 <= -self.sigma * (z ** 2):
                return lam
            k += 1
            if k > 20:  # Prevent infinite loop
                return lam

    def _coordinate_update(self, X, y, i, z, dii=None):
        d = self._newton_direction(X, y, i, dii)
        if abs(d) < 1e-12:
            return

        lam = 1.0
        if self.lambdas[i] < 1.0:  # Only do line search if needed
            lam = self._compute_lambda(X, y, i, d)

        delta = lam * d
        self.w[i] += delta

        # Efficient update of self.z for sparse column
        col_start = X.indptr[i]
        col_end = X.indptr[i + 1]
        indices = X.indices[col_start:col_end]
        data = X.data[col_start:col_end]

        self.z[indices] += delta * data



    def fit(self, X, y_labels):
        if not issparse(X) or not isinstance(X, csc_matrix):
            raise ValueError("X must be a CSC (Compressed Sparse Column) matrix")

        self.n = X.shape[1]
        self.w = np.zeros(self.n, dtype=np.float64)  # x0
        self.v = self.w.copy()
        self.z = X.dot(self.w)
        self.z_w = self.z.copy()
        self.z_v = X @ self.v 
        gamma_prev = 0.0

        for k in range(self.max_iter):
            # -- 1. Wyznacz gamma_k rozwiązując równanie kwadratowe -- check
            # gamma_prev = γ_{k-1}
            B = (self.sigma * gamma_prev**2 - 1) / self.n
            C = -gamma_prev**2

            discriminant = B**2 - 4 * C
            gamma_k = (-B + np.sqrt(discriminant)) / 2
            
            # -- 2. Oblicz alfa, beta -- check
            alpha_k = (self.n - gamma_k * self.sigma) / (gamma_k * (self.n**2 - self.sigma))
            beta_k = 1 - (gamma_k * self.sigma) / self.n

            # -- 3. y_k = α_k * v + (1 - α_k) * w -- check
            y = alpha_k * self.v + (1 - alpha_k) * self.w
            z_y = alpha_k * self.z_v + (1 - alpha_k) * self.z_w
            # -- 4. Wybierz losowy indeks --
            print(datetime.datetime.now())
            i = np.random.randint(0, self.n)
            print(datetime.datetime.now())
            #xji = X[:, i]

            col_start = X.indptr[i]
            col_end = X.indptr[i + 1]
            row_indices = X.indices[col_start:col_end]  # wiersze niezerowych elementów kolumny i
            data = X.data[col_start:col_end] 

            # -- 5. Licz gradient po y_k (czyli po ∇f(y)[i]) --
            margins = 1 - y_labels  * z_y
            active = margins > 0

            mask = active[row_indices]

            if not np.any(mask):
                grad_i = self.w[i]
            else:
                filtered_y = y[row_indices][mask]
                filtered_margins = margins[row_indices][mask]
                filtered_data = data[mask]
                grad_sum = np.sum(filtered_data * filtered_y * filtered_margins)
                grad_i = self.w[i] - 2 * self.C * grad_sum

            # -- 6. Lipschitz dla i-tego kierunku (aproksymacja Hessianu) --
            dii = self._d_double_prime_i_0(X, y_labels, i)

            # -- 7. Krok po i-tej współrzędnej (update x[i]) --
            delta_i = -grad_i / dii
            self.w[i] += delta_i
            self.z_w[row_indices] += delta_i * data

            # -- 8. Update v (momentum) --
            old_vi = self.v[i]
            self.v[i] = beta_k * self.v[i] + (1 - beta_k) * y[i] - (gamma_k / dii) * grad_i
            self.z_v[row_indices] += (self.v[i] - old_vi) * data


            # -- 9. Zaktualizuj gamma_prev --
            gamma_prev = gamma_k

            # -- 10. Monitoring co 100 iteracji --
            if k % 100 == 0:
                acc = self.score(X, y_labels)
                print(f"[{k:5d}] acc = {acc:.4f} , time = {datetime.datetime.now()}")

    def predict(self, X):
        return np.sign(X @ self.w)

    def score(self, X, y):
        return np.mean(self.predict(X) == y)
            
    def _objective(self, X, y):
        margins = 1 - y * self.z
        loss = np.sum((margins[margins > 0]) ** 2)
        return 0.5 * np.dot(self.w, self.w) + self.C * loss



In [ ]:
def load_svm_file(file_path, zero_based=True):
    labels = []
    rows = []
    cols = []
    data = []

    with open(file_path, 'r') as f:
        for i, line in enumerate(f):
            parts = line.strip().split()
            labels.append(float(parts[0]))

            for feat in parts[1:]:
                idx, val = feat.split(':')
                idx = int(idx) - (0 if zero_based else 1)
                rows.append(i)
                cols.append(idx)
                data.append(float(val))

    # Jawna konwersja do CSC
    from scipy.sparse import coo_matrix
    X = coo_matrix((data, (rows, cols))).tocsc()
    y = np.array(labels)

    return X, y
X,y =load_svm_file('../data/paper_data/news20.binary')


In [15]:
model = SparseCoordinateDescentSVM_2(C = 1, tol = 0, max_iter=3)
model.fit(X,y)

2025-06-09 17:22:38.303262
2025-06-09 17:22:38.303262
[    0] acc = 0.0000 , time = 2025-06-09 17:22:38.346248
2025-06-09 17:22:38.392251
2025-06-09 17:22:38.392251
2025-06-09 17:22:38.434247
2025-06-09 17:22:38.434247
